# LSTM Neural Network Fitting Analysis

## Preamble

The aim of this analysis is to develop a pipeline that consists of a LSTM Neural Network (LSTM NN) that given the opening stock price of a stock over the previous 40 days, can accurately predict the current opening stock price.  This project was inspired by the work done by Yacoub Ahmed that can be found here https://towardsdatascience.com/getting-rich-quick-with-machine-learning-and-stock-market-predictions-696802da94fe.  The aim is while using his original LSTM NN as a starting point, is to improve it and then leverage it to create a simulation of some stock trading on a few automobile stocks.  In the interest of academic honesty, it must be noted that while we took our inspiration from Yacoub and used his original LSTM model, all code used here is original.

Note, in call cases where we train a LSTM NN, we have commented it out and replaced it with code that loads in the models from memory.  This is done to save the markers time from having to retrain the models.  Which I certainly do not recommend unless you have a GPU at your disposal.

In [1]:
# Reading in neccessary packages
import yfinance as yf
from datetime import date, timedelta
import numpy as np
import pandas as pd
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
import tensorflow as tf
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf

ModuleNotFoundError: No module named 'yfinance'

In [ ]:
data = pd.read_csv('Data/ford.csv', index_col='Date')

## Check for Reasonable Predictor

To begin, we will build our model on Ford stock from January 1st, 2000 to November 17, 2020 (The day I am creating this).  Lets first as a formality check that the daily opening stock price is actually correlated to the previous days opening stock price.  To check this, we will create a plot of the sample correlation (ACF) below.  If we examine the AF plot below, we can clearly see that the opening price is highly correlated to the previous days opening stock price.  We can actually see this continues for many lags k.  Of course the reader may be wondering what we mean by lag k?  Let the opening stock price of ford be represented by the time series process ${Y_t}$.  The current days opening stock price we are interested in can be denated as $Y_t$.  Then in the plot below we can a high correlation between $Y_t$ and $Y_{t-k}$ where k is some integer k that denotes some number of units back in time.  For instance, from the plot below we can see that the sample ACF between $Y_t$ and $Y_{t-2}$ produces a sample ACF very close to 1.  Moving on, our conclusion from this plot is simply that the current opening stock price of Ford has a high correlation with its past opening stock prices.  Thus, it could be a strong predictor.  Lastly, the question remains how far back in the past to predict the current opening stock price.  That is many many past days should we use to predict the current day.  To choose this we arbitrarilty chose 4.

In [ ]:
# CITATION: https://medium.com/@krzysztofdrelczuk/acf-autocorrelation-function-simple-explanation-with-python-example-492484c32711
plot_acf(data['Open'], lags=100, title='Sample ACF of Ford Opening Day Stock Price')

# NOTE: for some reason this produces 2 of the same plot, it could not be removed.

## Set Up Code

The following few 4 cells contain code that will be use to create, and evaluate the models

In [ ]:
# This defines the numpy of days in the past we will use to predict the current price
LAG = 40

In [ ]:
# Plots the results of a mode.
def plot(yHat, y, title):
    plt.plot(y)
    plt.plot(yHat)
    plt.legend(['Real', 'Predicted'])
    plt.title(title)
    plt.show()
    

In [ ]:
# Calculates the MSE
def mse(x, y):
    preds = model.predict(x)
    preds = scaler.inverse_transform(preds)
    unscaled_yTest = scaler.inverse_transform(np.reshape(y, (-1, 1)))
    return np.mean(np.square(preds-unscaled_yTest)), preds, unscaled_yTest

In [ ]:
# This function accepts the raw data and transforms it into the form required for the LSTM NN.  
# - It removes unneccessary columns
# - scales the data between 0 and 1
# - transforms into numpy arrays
def transform(df, n = 50):
    df = pd.DataFrame(df['Open'])    # Removing all the other columns as we are only predicting if we should buy based on the opening stock price

    # Normalizing to 0-1 range
    scaler = MinMaxScaler(feature_range=(0,1)) 
    scaler.fit(df)
    df = pd.DataFrame(scaler.transform(df),columns=df.columns, index = pd.to_datetime(df.index))
    
    # Creating 40 columns that give the past 40 day opening stock price
    for i in range(1,n+1):
        df[f'Open-{i}'] = df['Open'].shift(-i)
    
    # Subsetting for the neccessary columns
    df = df.iloc[0:-n,0:]
    
    # Splitting into training and testing data (test size is about last 2 years)
    dt = pd.to_datetime(date(2020, 11, 17) - timedelta(days=730))
    train = df[df.index < dt]
    test = df[df.index >= dt]
    
    # Splitting into appropriat x and y values
    xTrain = train.iloc[:,1:]
    yTrain = train.iloc[:,0]
    xTest = test.iloc[:,1:]
    yTest = test.iloc[:,0]
    
    
    # Converting to numpy arrays to feed into model
    xTrain = xTrain.to_numpy()
    yTrain = yTrain.to_numpy()
    xTest = xTest.to_numpy()
    yTest = yTest.to_numpy()
    
    # Reshaping to get correct form
    xTrain = np.reshape(xTrain, (xTrain.shape[0], xTrain.shape[1], 1))
    xTest = np.reshape(xTest, (xTest.shape[0], xTest.shape[1], 1))
    
    return xTrain, yTrain, xTest, yTest, scaler    

In [ ]:
xTrain, yTrain, xTest, yTest, scaler = transform(data, LAG)

## First Model
We have now transformed the data into the required format with both a training and testing.  So lets try fitting to a model!

In [ ]:
# CITATION 1: https://www.youtube.com/watch?v=BSpXCRTOLJA
# CITATION 2: https://towardsdatascience.com/getting-rich-quick-with-machine-learning-and-stock-market-predictions-696802da94fe
# Citation 1 was used as a starting point to get the model to make at least semi-accurate predictions.
# model = Sequential()
# model.add(LSTM(128,input_shape=(xTrain.shape[1],xTrain.shape[2])))
# model.add(Dropout(0.2))
# model.add(Dense(64))
# model.add(Activation('sigmoid'))
# model.add(Dense(1))
# model.add(Activation('linear'))

# opt = tf.keras.optimizers.Adam(lr=0.0005)
# model.compile(optimizer=opt, loss='mse')
# model.fit(xTrain,yTrain, epochs=50, validation_data=(xTest,yTest))

# CITATION: https://machinelearningmastery.com/save-load-keras-deep-learning-models/
# load json and create model
json_file = open('data/fordLSTM_50EPOCH.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("data/fordLSTM_50EPOCH.h5")
print("Loaded model from disk")



In [ ]:
meanSquared, yHat, y = mse(xTrain,yTrain)
plot(yHat, y, 'Training Data')
print("MSE:",meanSquared)

In [ ]:
mse(xTest,yTest)
meanSquared, yHat, y = mse(xTest,yTest)
plot(yHat, y, 'Training Data')
print("MSE:",meanSquared)

We can see the results of our model above.  We can see our model begin to take the rough shape of the actual data on the Test Data.  However, we need it to be far closer.  As we will see shortly, the model is in fact very undertrained.  To adjust for this, lets run the model on varying Epochs (i.e. how many times you run the data through to train the model).  We will test this on 50,100,500,1000 and 2000 epochs and record the MSE to give a rough evaluation of the models.

## Model Tuning 1: Epochs

In [ ]:
# def epochs(epoch):
#     model = Sequential()
#     model.add(LSTM(128,input_shape=(xTrain.shape[1],xTrain.shape[2])))
#     model.add(Dropout(0.2))
#     model.add(Dense(64))
#     model.add(Activation('sigmoid'))
#     model.add(Dense(1))
#     model.add(Activation('linear'))

#     opt = tf.keras.optimizers.Adam(lr=0.0005)
#     model.compile(optimizer=opt, loss='mse')
#     model.fit(xTrain,yTrain, epochs=epoch, validation_data=(xTest,yTest))
#     return model

# res = pd.DataFrame(columns=['epoch','mseTest','mseTrain'])
# for i in [50,100,500,1000,2000]:
#     model = epochs(i)
#     meanSquaredTest, yHat, y = mse(xTest,yTest)
#     meanSquaredTrain, yHat, y = mse(xTrain,yTrain)
#     res = res.append({'epoch':i,'mseTest':meanSquaredTest,'mseTrain':meanSquaredTrain},ignore_index=True)

resultsEpoch = pd.read_csv('Data/resultsEpoch.csv')
resultsEpoch

Above we see the results of training the modle further.  We can clearly see that MSE decreases as we continue to train the model.  However, when we reach 3000 epochs the MSE of the Testing data begins to increase and the MSE of the Training Data begins to decrease.  Is is possible that we are overfitting the data at 3000 epochs.  Thus, we will use 2000 epochs in our model.  Below we vizualize the result of the 2000 epoch model.

In [ ]:
# CITATION: https://machinelearningmastery.com/save-load-keras-deep-learning-models/
json_file = open('data/fordLSTM1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("data/fordLSTM1.h5")
print("Loaded model from disk")  


In [ ]:
plot(yHat, y, 'Training Data')
meanSquared, yHat, y = mse(xTrain,yTrain)
print("MSE:",meanSquared)

In [ ]:
plot(yHat, y, 'Test Data')
meanSquared, yHat, y = mse(xTest,yTest)
print("MSE:",meanSquared)

Above are the plots of our test and training data.  The results appear to be significantly improved considering we have not done any formal tuning at this point.  Surprisingly, we can actually see that that model appears to perform better on the testing data than the training data.  We take this as an indication that the model does not appear to be over fitting the training data.  Thus, no further dropouts will be added to the model nor will any adjustments be made there.  However, we will continue to try and improve the model. 

Note, the reader may notice that the MSE of the 2000 epoch model displayed with their respective plots differs from the MSE of the 2000 epoch model displayed in the tabel above.  This is likely due to where the MSE's were computed.  While the code was the same, the MSE in the table above was computed on Google Colab while training the models and the MSE computed with the plots was done locally.  There is perhaps some difference of rounding.

## Model Tuning 2: Neurons
Lets now tune the number of hidden nodes contained in the LSTM layer.  Note, the original value of 128 was chosen arbitrarilty.  To test this, we will do so on a model with 500 epochs to minimize computation time.  Furthermore, not that the range of neurons chosen is in fact arbitraty.

In [ ]:
def epochs(neurons):
    model = Sequential()
    model.add(LSTM(neurons,input_shape=(xTrain.shape[1],xTrain.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(64))
    model.add(Activation('sigmoid'))
    model.add(Dense(1))
    model.add(Activation('linear'))

    opt = tf.keras.optimizers.Adam(lr=0.0005)
    model.compile(optimizer=opt, loss='mse')
    model.fit(xTrain,yTrain, epochs=500, validation_data=(xTest,yTest))
    return model

res = pd.DataFrame(columns=['epoch','mseTest','mseTrain'])
for i in [50,70,90,100,120,140]:
    model = epochs(i)
    meanSquaredTest, yHat, y = mse(xTest,yTest)
    meanSquaredTrain, yHat, y = mse(xTrain,yTrain)
    res = res.append({'epoch':i,'mseTest':meanSquaredTest,'mseTrain':meanSquaredTrain},ignore_index=True)